WEEK 1

In [1]:
from statistics import mean
import numpy as np

In [2]:
# Calculating the mean and median of a dataset

def calc_stats(data):
  data = np.loadtxt(data, delimiter = ',')
  mean = np.mean(data)
  median = np.median(data)
  return (np.round(mean,1),np.round(median,1))

In [3]:
# Write a mean_datasets function that reads a list of CSV files and returns an array of the mean of each cell in data.

import numpy as np


def mean_datasets(filenames):
  n = len(filenames)
  if n > 0:
    data = np.loadtxt(filenames[0],delimiter = ',')
    for i in range(1,n):
      data += np.loadtxt(filenames[i], delimiter = ',')
      
      data_mean = data/n
      
      return np.round(data_mean,1)

In [4]:
# Finding brightest pixel in a FITS file

from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt

def load_fits(file):
  hdulist = fits.open(file)                      #HDU = header/data units
  data = hdulist[0].data
  
  bright = np.argmax(data)                       #argmax gets the maximum of the array
  max_pos = np.unravel_index(bright, data.shape) #unravel_undex does this based on array dimensions
  return max_pos

In [1]:
# Mean of a set of FITS files

import numpy as np
from astropy.io import fits

def mean_fits(filenames):
  n = len(filenames)
  if n>0:
    hdulist = fits.open(filenames[0])
    data = hdulist[0].data
    hdulist.close()
    
    for i in range(1,n):
      hdulist = fits.open(filenames[i])
      data += hdulist[0].data
      hdulist.close()
  
    mean = data/n
    return mean

Week 2

In [2]:
#Write a find_closest function that takes a catalogue and the position of a target source (a right ascension 
        # and declination) and finds the closest match for the target source in the catalogue.

import numpy as np

def hms2dec(hr, m, s):
  dec = hr + m/60 + s/3600
  return dec*15

def dms2dec(d, m, s):
  sign = d/abs(d)
  dec = abs(d) + m/60 + s/3600
  return sign*dec

def import_bss():
  res = []
  data = np.loadtxt('bss.dat', usecols=range(1, 7))
  for i, row in enumerate(data, 1):
    res.append((i, hms2dec(row[0], row[1], row[2]), dms2dec(row[3], row[4], row[5])))
  return res

def import_super():
  data = np.loadtxt('super.csv', delimiter=',', skiprows=1, usecols=[0, 1])
  res = []
  for i, row in enumerate(data, 1):
    res.append((i, row[0], row[1]))
  return res

def angular_dist(ra1, dec1, ra2, dec2):
  # Convert to radians
  r1 = np.radians(ra1)
  d1 = np.radians(dec1)
  r2 = np.radians(ra2)
  d2 = np.radians(dec2)

  a = np.sin(np.abs(d1 - d2)/2)**2
  b = np.cos(d1)*np.cos(d2)*np.sin(np.abs(r1 - r2)/2)**2

  angle = 2*np.arcsin(np.sqrt(a + b))
    
  # Convert back to degrees
  return np.degrees(angle)

def find_closest(cat, ra, dec):
  min_dist = np.inf
  min_id = None
  for id1, ra1, dec1 in cat:
    dist = angular_dist(ra1, dec1, ra, dec)
    if dist < min_dist:
      min_id = id1
      min_dist = dist
    
  return min_id, min_dist

In [1]:
#Crossmatching two catalogues using a maximum distance

import numpy as np

def angular_dist(RA1, dec1, RA2, dec2):
    # Convert to radians
    r1 = np.radians(RA1)
    d1 = np.radians(dec1)
    r2 = np.radians(RA2)
    d2 = np.radians(dec2)
    
    deltar = np.abs(r1 - r2)
    deltad = np.abs(d1 - d2)
    angle = 2*np.arcsin(np.sqrt(np.sin(deltad/2)**2 
                        + np.cos(d1)*np.cos(d2)*np.sin(deltar/2)**2))
    
    # Convert back to degrees
    return np.degrees(angle)

def crossmatch(cat1, cat2, max_radius):
    matches = []
    no_matches = []
    for id1, ra1, dec1 in cat1:
        closest_dist = np.inf
        closest_id2 = None
        for id2, ra2, dec2 in cat2:
            dist = angular_dist(ra1, dec1, ra2, dec2)
            if dist < closest_dist:
                closest_id2 = id2
                closest_dist = dist
        
        # Ignore match if it's outside the maximum radius
        if closest_dist > max_radius:
            no_matches.append(id1)
        else:
            matches.append((id1, closest_id2, closest_dist))

    return matches, no_matches